In [1]:
import pandas as pd



News_videos = "News_person_search_results.csv"
Entertainment_videos = "Other_Entertainment_person_search_results.csv"
UK_Show_videos= "UK_Shows_person_search_results.csv"
US_Show_videos="US_Shows_person_search_results.csv"


News_sources_and_persons = []
Entertainment_sources_and_persons = []
UK_sources_and_persons = []
US_sources_and_persons = []

News_urls = []
Entertainment_urls = []
UK_urls = []
US_urls = []
def SaveFunction(file, url_list, sources_and_persons):
    df = pd.read_csv(file, dtype = {
        'video_id': str,
        'url': str,
        'title': str,
        'channel_name': str,
        'published_at': str,
        'view_count': 'Int64',
        'persons_found': str,
        'total_persons': 'Int64'
    })
    df['persons_found_list'] = (df['persons_found'].fillna('')
                     .apply(lambda x: [name.strip() for name in x.split(',') if name.strip()])
                    )
    for url, source, persons in zip(df['url'], df['video_id'], df['persons_found_list']):
        url_list.append(url)
        record = {
            "source": source,
            "persons_found": persons
        }
        
        sources_and_persons.append(record)

SaveFunction(News_videos, News_urls, News_sources_and_persons)
SaveFunction(Entertainment_videos, Entertainment_urls, Entertainment_sources_and_persons)
SaveFunction(UK_Show_videos, UK_urls, UK_sources_and_persons)
SaveFunction(US_Show_videos, US_urls, US_sources_and_persons)

print(f"We have {len(News_urls)} News videos to process.")
print(f"We have {len(Entertainment_urls)} Entertainment videos to process.")
print(f"We have {len(UK_urls)} UK Show videos to process.")
print(f"We have {len(US_urls)} US Show videos to process.")
print(f"In total, there are {len(News_urls)+len(Entertainment_urls)+len(UK_urls)+ len(US_urls)} videos.")
print(f"Sample persons_found list: {UK_sources_and_persons[:3]}")

We have 22716 News videos to process.
We have 404 Entertainment videos to process.
We have 518 UK Show videos to process.
We have 3200 US Show videos to process.
In total, there are 26838 videos.
Sample persons_found list: [{'source': 'LOA-rEMU8pA', 'persons_found': ['Ben Affleck', 'Margot Robbie', 'Ryan Gosling']}, {'source': 't9H3_ewSKI8', 'persons_found': ['Chris Hemsworth', 'Hugh Jackman', 'Tom Cruise']}, {'source': 'rFnB7gTk8L8', 'persons_found': ['Nicole Kidman', 'Ryan Gosling']}]


In [8]:
import os
import time
import random
import logging
import json
from datetime import datetime
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from pathlib import Path

from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing

# ─── CONFIG ────────────────────────────────────────────────────────────────
RETRIES       = 3
BASE_DELAY    = 2.0
STATE_FILE    = "US_Shows_yt_state.json"
SUMMARY_FILE  = "US_Shows_summary.json"
PAUSE_SECONDS = 1.5

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

if os.path.exists(STATE_FILE):
    state = json.loads(Path(STATE_FILE).read_text(encoding="utf-8"))
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = set(state.get("not_working", []))
    processed_urls = set(state.get("processed_urls", []))
    logging.info(
        f"Resumed state: {len(all_transcripts)} docs, "
        f"{len(not_working)} failures, {len(processed_urls)} processed"
    )
else:
    all_transcripts, not_working, processed_urls = [], set(), set()


def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working":    sorted(not_working),
        "processed_urls": sorted(processed_urls),
    }
    Path(STATE_FILE).write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")


from typing import List
from langchain.schema import Document
def load_with_retry(url: str) -> List[Document]:
    """
    1) Try official subtitles (translation=None)
    2) Fall back to auto-generated English (translation="en")
    3) On parse/network errors retry both steps with backoff
    """
    for attempt in range(1, RETRIES + 1):
        # official subtitles
        try:
            docs = YoutubeLoader.from_youtube_url(
                url, add_video_info=True, translation=None
            ).load()
            if docs:
                logging.info(f"[{url}] official transcript loaded (attempt {attempt})")
                return docs
            logging.info(f"[{url}] no official subtitles found (attempt {attempt})")
        except (ExpatError, ParseError) as e:
            logging.warning(f"[{url}] official parse error (attempt {attempt}): {e}")
        except Exception as e:
            logging.error(f"[{url}] unexpected official loader error: {e}", exc_info=True)

        # auto-generated English fallback
        try:
            docs = YoutubeLoader.from_youtube_url(
                url, add_video_info=True, translation="en"
            ).load()
            if docs:
                logging.info(f"[{url}] auto-en transcript loaded (attempt {attempt})")
                return docs
            logging.warning(f"[{url}] no auto-en subtitles found (attempt {attempt})")
        except (ExpatError, ParseError) as e:
            logging.warning(f"[{url}] auto-en parse error (attempt {attempt}): {e}")
        except Exception as e:
            logging.error(f"[{url}] unexpected auto-en loader error: {e}", exc_info=True)

        # backoff before retry
        backoff = BASE_DELAY * (2 ** (attempt - 1)) + random.random()
        logging.info(f"[{url}] retrying in {backoff:.1f}s…")
        time.sleep(backoff)

    logging.error(f"[{url}] failed after {RETRIES} attempts")
    return []

# ─── MAIN LOOP ─────────────────────────────────────────────────────────────
def process_urls(urls, skip_already_processed=True):
    for url in urls:
        # Only skip if we're in the first-pass mode
        if skip_already_processed and url in processed_urls:
            continue

        docs = load_with_retry(url)
        if docs:
            all_transcripts.extend(docs)
            not_working.discard(url)
        else:
            not_working.add(url)
        processed_urls.add(url)
        save_state()
        time.sleep(PAUSE_SECONDS)

# ─── FIRST PASS: skip URLs we've already done ─────────────────────────────
process_urls(US_urls, skip_already_processed=True)

# ─── RETRY PASS: force all failures through ────────────────────────────────
if not_working:
    logging.info(f"Retrying {len(not_working)} previously failed URLs...")
    process_urls(list(not_working), skip_already_processed=False)

2025-06-16 17:32:17 INFO     Resumed state: 32 docs, 18 failures, 50 processed
2025-06-16 17:32:20 ERROR    [https://youtu.be/NiIimKTo0Pw] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=NiIimKTo0Pw! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (NiIimKTo0Pw) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")

(GENERATED)
None

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Traceback (most recent call l

2025-06-16 17:32:24 INFO     [https://youtu.be/NiIimKTo0Pw] retrying in 2.8s…
2025-06-16 17:32:30 ERROR    [https://youtu.be/NiIimKTo0Pw] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=NiIimKTo0Pw! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (NiIimKTo0Pw) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")

(GENERATED)
None

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Traceback (most recent call la

2025-06-16 17:32:33 INFO     [https://youtu.be/NiIimKTo0Pw] retrying in 4.9s…
2025-06-16 17:32:41 ERROR    [https://youtu.be/NiIimKTo0Pw] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=NiIimKTo0Pw! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (NiIimKTo0Pw) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")

(GENERATED)
None

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Traceback (most recent call la

2025-06-16 17:32:45 INFO     [https://youtu.be/NiIimKTo0Pw] retrying in 8.1s…
2025-06-16 17:32:53 ERROR    [https://youtu.be/NiIimKTo0Pw] failed after 3 attempts
2025-06-16 17:32:58 ERROR    [https://youtu.be/ZZ9LyrupNIg] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=ZZ9LyrupNIg! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (ZZ9LyrupNIg) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")

(GENERATED)
None

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there ar

2025-06-16 17:33:01 INFO     [https://youtu.be/ZZ9LyrupNIg] retrying in 2.6s…
2025-06-16 17:33:07 ERROR    [https://youtu.be/ZZ9LyrupNIg] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=ZZ9LyrupNIg! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (ZZ9LyrupNIg) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")

(GENERATED)
None

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Traceback (most recent call la

...

2025-06-16 18:29:28 INFO     [https://youtu.be/5ZrIxIN99hI] retrying in 8.9s…
2025-06-16 18:29:37 ERROR    [https://youtu.be/5ZrIxIN99hI] failed after 3 attempts
2025-06-16 18:29:42 ERROR    [https://youtu.be/26p2BP21Ra8] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=26p2BP21Ra8! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxies to hide your IP address, as explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocke

2025-06-16 18:29:45 INFO     [https://youtu.be/26p2BP21Ra8] retrying in 2.6s…
2025-06-16 18:29:51 ERROR    [https://youtu.be/26p2BP21Ra8] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=26p2BP21Ra8! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (26p2BP21Ra8) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")

(GENERATED)
None

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Traceback (most recent call la

2025-06-16 18:29:54 INFO     [https://youtu.be/26p2BP21Ra8] retrying in 5.0s…
2025-06-16 18:30:03 ERROR    [https://youtu.be/26p2BP21Ra8] unexpected official loader error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=26p2BP21Ra8! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (26p2BP21Ra8) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")

(GENERATED)
None

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Traceback (most recent call la

2025-06-16 18:30:06 INFO     [https://youtu.be/26p2BP21Ra8] retrying in 8.2s…
2025-06-16 18:30:14 ERROR    [https://youtu.be/26p2BP21Ra8] failed after 3 attempts


KeyboardInterrupt: 

In [ ]:
def summarize():
    failed    = not_working
    completed = sorted(processed_urls - failed)
    remaining = sorted(set(News_urls) - processed_urls)

    summary = {
        "completed_urls": completed,
        "failed_urls":    sorted(failed),
        "remaining_urls": remaining,
    }
    Path(SUMMARY_FILE).write_text(
        json.dumps(summary, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    logging.info(
        f"✅ Completed: {len(completed)}, "
        f"❌ Failed: {len(failed)}, "
        f"⏳ Remaining: {len(remaining)}"
    )

summarize()

In [ ]:
import os
import time
import random
import logging
import json
from datetime import datetime
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from pathlib import Path

from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing

# ─── CONFIG ────────────────────────────────────────────────────────────────
RETRIES       = 3
BASE_DELAY    = 2.0
STATE_FILE    = "Entertainment_yt_state.json"
SUMMARY_FILE  = "Entertainment_summary.json"
PAUSE_SECONDS = 1.5

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

if os.path.exists(STATE_FILE):
    state = json.loads(Path(STATE_FILE).read_text(encoding="utf-8"))
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = set(state.get("not_working", []))
    processed_urls = set(state.get("processed_urls", []))
    logging.info(
        f"Resumed state: {len(all_transcripts)} docs, "
        f"{len(not_working)} failures, {len(processed_urls)} processed"
    )
else:
    all_transcripts, not_working, processed_urls = [], set(), set()

In [ ]:
def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working":    sorted(not_working),
        "processed_urls": sorted(processed_urls),
    }
    Path(STATE_FILE).write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")




In [ ]:
# ─── FIRST PASS: skip URLs we've already done ─────────────────────────────
process_urls(Entertainment_urls, skip_already_processed=True)

# ─── RETRY PASS: force all failures through ────────────────────────────────
if not_working:
    logging.info(f"Retrying {len(not_working)} previously failed URLs...")
    process_urls(list(not_working), skip_already_processed=False)


In [ ]:
def summarize():
    failed    = not_working
    completed = sorted(processed_urls - failed)
    remaining = sorted(set(Entertainment_urls) - processed_urls)

    summary = {
        "completed_urls": completed,
        "failed_urls":    sorted(failed),
        "remaining_urls": remaining,
    }
    Path(SUMMARY_FILE).write_text(
        json.dumps(summary, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    logging.info(
        f"✅ Completed: {len(completed)}, "
        f"❌ Failed: {len(failed)}, "
        f"⏳ Remaining: {len(remaining)}"
    )

summarize()


In [ ]:
import os
import time
import random
import logging
import json
from datetime import datetime
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from pathlib import Path

from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing

# ─── CONFIG ────────────────────────────────────────────────────────────────
RETRIES       = 3
BASE_DELAY    = 2.0
STATE_FILE    = "UK_Shows_yt_state.json"
SUMMARY_FILE  = "UK_Shows_summary.json"
PAUSE_SECONDS = 1.5

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

if os.path.exists(STATE_FILE):
    state = json.loads(Path(STATE_FILE).read_text(encoding="utf-8"))
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = set(state.get("not_working", []))
    processed_urls = set(state.get("processed_urls", []))
    logging.info(
        f"Resumed state: {len(all_transcripts)} docs, "
        f"{len(not_working)} failures, {len(processed_urls)} processed"
    )
else:
    all_transcripts, not_working, processed_urls = [], set(), set()

In [ ]:
def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working":    sorted(not_working),
        "processed_urls": sorted(processed_urls),
    }
    Path(STATE_FILE).write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")



In [ ]:
# ─── FIRST PASS: skip URLs we've already done ─────────────────────────────
process_urls(UK_urls, skip_already_processed=True)

# ─── RETRY PASS: force all failures through ────────────────────────────────
if not_working:
    logging.info(f"Retrying {len(not_working)} previously failed URLs...")
    process_urls(list(not_working), skip_already_processed=False)


In [ ]:
def summarize():
    failed    = not_working
    completed = sorted(processed_urls - failed)
    remaining = sorted(set(UK_urls) - processed_urls)

    summary = {
        "completed_urls": completed,
        "failed_urls":    sorted(failed),
        "remaining_urls": remaining,
    }
    Path(SUMMARY_FILE).write_text(
        json.dumps(summary, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    logging.info(
        f"✅ Completed: {len(completed)}, "
        f"❌ Failed: {len(failed)}, "
        f"⏳ Remaining: {len(remaining)}"
    )

summarize()


In [ ]:
import os
import time
import random
import logging
import json
from datetime import datetime
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from pathlib import Path

from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing

# ─── CONFIG ────────────────────────────────────────────────────────────────
RETRIES       = 3
BASE_DELAY    = 2.0
STATE_FILE    = "US_Shows_yt_state.json"
SUMMARY_FILE  = "US_Shows_summary.json"
PAUSE_SECONDS = 1.5

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

if os.path.exists(STATE_FILE):
    state = json.loads(Path(STATE_FILE).read_text(encoding="utf-8"))
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = set(state.get("not_working", []))
    processed_urls = set(state.get("processed_urls", []))
    logging.info(
        f"Resumed state: {len(all_transcripts)} docs, "
        f"{len(not_working)} failures, {len(processed_urls)} processed"
    )
else:
    all_transcripts, not_working, processed_urls = [], set(), set()

In [ ]:
def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working":    sorted(not_working),
        "processed_urls": sorted(processed_urls),
    }
    Path(STATE_FILE).write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")



In [ ]:
# ─── FIRST PASS: skip URLs we've already done ─────────────────────────────
process_urls(US_urls, skip_already_processed=True)

# ─── RETRY PASS: force all failures through ────────────────────────────────
if not_working:
    logging.info(f"Retrying {len(not_working)} previously failed URLs...")
    process_urls(list(not_working), skip_already_processed=False)

In [ ]:
def summarize():
    failed    = not_working
    completed = sorted(processed_urls - failed)
    remaining = sorted(set(US_urls) - processed_urls)

    summary = {
        "completed_urls": completed,
        "failed_urls":    sorted(failed),
        "remaining_urls": remaining,
    }
    Path(SUMMARY_FILE).write_text(
        json.dumps(summary, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    logging.info(
        f"✅ Completed: {len(completed)}, "
        f"❌ Failed: {len(failed)}, "
        f"⏳ Remaining: {len(remaining)}"
    )

summarize()